In [11]:
import io
import os
from google.cloud import speech
from google.cloud.speech import enums
from google.cloud.speech import types
import pandas as pd
from konlpy.tag import Twitter
from collections import Counter 
from pydub import AudioSegment
from pydub.playback import play
import argparse
from moviepy.editor import *
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import time
import pyaudio
import wave
from array import array

# 형태소 자르는것
twitter = Twitter()

xx = pd.read_csv("욕욕.csv",names = ["욕"],encoding='cp949')
vect = CountVectorizer(tokenizer=twitter.nouns)
vect.fit(xx['욕']) # 욕사전 학습
vect.vocabulary_ # 욕사전 확인

#api json파일 설정
GOOGLE_APPLICATION_CREDENTIALS = "voice-be4b7165be4c.json"

# FORMAT=pyaudio.paInt16
CHANNELS=2
# RATE=44100
# CHUNK=1024
# RECORD_SECONDS=15
# FILE_NAME="RECORDING.wav"

# audio=pyaudio.PyAudio() #instantiate the pyaudio

# #recording prerequisites
# stream=audio.open(format=FORMAT,channels=CHANNELS,
#                   rate=RATE,
#                   input=True,
#                   frames_per_buffer=CHUNK)

# #starting recording
# frames=[]

# for i in range(0,int(RATE/CHUNK*RECORD_SECONDS)):
#     data=stream.read(CHUNK)
#     data_chunk=array('h',data)
#     vol=max(data_chunk)
#     if(vol>=500):
#         print("something said")
#         frames.append(data)
#     else:
#         print("nothing")
#     print("\n")


# #end of recording
# stream.stop_stream()
# stream.close()
# audio.terminate()
# #writing to file
# wavfile=wave.open(FILE_NAME,'wb')
# wavfile.setnchannels(CHANNELS)
# wavfile.setsampwidth(audio.get_sample_size(FORMAT))
# wavfile.setframerate(RATE)
# wavfile.writeframes(b''.join(frames))#append frames recorded to file


client = speech.SpeechClient()
videoclip = VideoFileClip("chulgoo4.mp4")
audioclip = videoclip.audio
audioclip.to_audiofile('chulgoo4_음성.wav')
wav_file = "chulgoo4_음성.wav"
# wav_file = "RECORDING.wav"

with io.open(wav_file, 'rb') as audio_file:
    content = audio_file.read()            

audio = types.RecognitionAudio(content=content)
config = types.RecognitionConfig(
    encoding=enums.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=44100,audio_channel_count=CHANNELS,
    language_code='ko-KR',
    enable_word_time_offsets=True)

operation = client.long_running_recognize(config, audio)


print('Waiting for operation to complete...')
result = operation.result(timeout=90)
    
word_list = []
start_time_list = []
end_time_list = []

for k in result.results:
    alternatives = k.alternatives
    for alternative in alternatives:
        print('Transcript: {}'.format(alternative.transcript))
        print('Confidence: {}'.format(alternative.confidence))
    
    for word_info in alternative.words:
        wo = word_info.word
        word_list.append(wo)
        start_time = word_info.start_time
        end_time = word_info.end_time
        start_time_list.append(start_time.seconds + start_time.nanos * 1e-9)
        end_time_list.append(end_time.seconds + end_time.nanos * 1e-9)
        print('Word: {}, start_time: {}, end_time: {}'.format(
            wo,
            start_time.seconds + start_time.nanos * 1e-9,
            end_time.seconds + end_time.nanos * 1e-9))
        
sentences_tag = []
for sentence in word_list:
    morph = twitter.pos(sentence)
    sentences_tag.append(morph)

noun_adj_list = []
for sentence1 in sentences_tag:
    for word, tag in sentence1:
        if tag in ['Noun','Adjective']:
            noun_adj_list.append(word)

word_vect = vect.transform(noun_adj_list)
feature_list = []

for i in word_vect.toarray():
    for j in range(0,len(i)):
        if i[j] == 1:
            feature_list.append(vect.get_feature_names()[j])

            
sound1 = AudioSegment.from_file(wav_file)


for i in range(0,len(word_list)):
    for j in range(0,len(feature_list)):
        if feature_list[j] in word_list[i]:
            print(start_time_list[i])
            print(end_time_list[i])
            sound2 = AudioSegment.from_file("Beep2.wav") 
            if end_time_list[i]-start_time_list[i]>2.5:
                sound2 = sound2[start_time_list[i]*1000:(end_time_list[i]*1000)]
                sound1 = sound1.overlay(sound2,position=(1000*start_time_list[i])+2500,gain_during_overlay =-100)
            else :
                sound2 = sound2[start_time_list[i]*1000:(end_time_list[i]*1000)]
                sound1 = sound1.overlay(sound2,position=1000*start_time_list[i],gain_during_overlay =-100)
            


sound1.export(wav_file, format='wav')

videoclip2 = videoclip.set_audio(AudioFileClip(wav_file))
videoclip2.write_videofile('Chulgoo4_filter.avi',codec='libx264')

MoviePy - Writing audio in %s


MoviePy - Done.
Waiting for operation to complete...
Transcript: 꿈에서요 거짓말 아니다 근데 내 모발이 딱 나오더니만 안아 보고 의자를 개새끼 군대 영장 받았습니다 다음에 입대하는 꿈
Confidence: 0.8356379270553589


TypeError: No matching overloads found for kr.lucypark.okt.OktInterface.tokenize(google.protobuf.pyext._message.RepeatedCompositeContainer,java.lang.Boolean,java.lang.Boolean), options are:
	public java.util.List kr.lucypark.okt.OktInterface.tokenize(java.lang.String,java.lang.Boolean,java.lang.Boolean)

	at JPMethod::findOverload(native\common\jp_method.cpp:242)
	at JPMethod::findOverload(native\common\jp_method.cpp:245)
	at JPMethod::invoke(native\common\jp_method.cpp:253)
	at PyJPMethod::__call__(native\python\pyjp_method.cpp:142)
